In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import geopandas as geo
import ipywidgets as widgets
from IPython.display import display
from mpl_toolkits.axes_grid1 import make_axes_locatable

%matplotlib inline

Import Dataset

In [ ]:
root = 'C:/Users/Aaron/PycharmProjects/ECE-225'
data_path = Path(root + '/data/output.csv')
df = pd.read_csv(data_path)

#### Import .shp file data for counties

In [ ]:
shp_counties = Path(root + "/data/UScounties/uscounties.shp")

c_map = geo.read_file(shp_counties)
c_map = c_map[c_map.STATE_NAME != 'Alaska']
c_map = c_map[c_map.STATE_NAME != 'Hawaii']
c_map.rename(columns={'NAME':'County', 'STATE_NAME':'State'}, inplace=True)

df.County.convert_dtypes()
c_map.County.convert_dtypes()


In [ ]:
c_map.head()

### Merge data with shp to gain geo poly

In [ ]:
# Clean Data and Merge
df['County'] = df.County.str.replace('County', '')
# df.insert(5, "geometry", 0, True)
#
# for i in range(len(df.County)):
#     dcounty = df.iloc[i].at['County']
#     dstate = df.iloc[i].at['State']
#     df.iloc[i].at['geometry'] = c_map.iloc[(c_map['County'] == dcounty) & (c_map['State'] == dstate)].at['geometry']

df1 = df[['County', 'State']]
df2 = c_map[['County','State','geometry']]
m = pd.merge(df1, df2, how='right',indicator=True)
geodata = geo.geodataframe.GeoDataFrame(m, geometry=m.geometry)
m.head(10)

In [ ]:
df['County'].value_counts().head(10)

In [ ]:
# c_map.sort_values(by='County', inplace=True)
test = c_map.drop(columns='geometry')
# test.head()
test.sort_values(by='County', inplace=True)
test.value_counts().head(10)
# test = c_map['counts'] = c_map['County'].map(df['County'].value_counts())
# test.head(10)

In [ ]:
print(np.array(df['County'].unique()).shape)
print(np.array(c_map['County'].unique()).shape)
print(geodata.shape)
print(df.shape)
print(m.shape)

### Plot with Slider

In [ ]:
months = ["January", "February", "March","April", "May", "June", "July", "August","September", "October","November" , "December" ]
def plotmap(y, m):
    fig, ax = plt.subplots(1, figsize=(12,12))
    div = make_axes_locatable(ax)
    cax = div.append_axes("right", size="5%", pad=0.1)
    c_map.boundary.plot(ax=ax)
    newdata = geodata[(geodata['Year'] == y) & (geodata['Month'] == m)]
    newdata.plot(column='Rate',ax=ax, legend=True, cax=cax)
    display(newdata.shape)
year_s = widgets.IntSlider(min=np.min(df.Year), max=np.max(df.Year), description="Year")
mon_s = widgets.Dropdown(options=months, description="Month", value="January")
widgets.interact(plotmap, y=year_s, m=mon_s)